In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import plotly_express as px
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from scipy.spatial import distance

SEED = 42

In [2]:
projects = pd.read_csv("../data/nrao_projects.csv")
projects = projects.set_index('project_code')

In [3]:
projects.head()

,project_title,project_abstract,fs_type,target,raw_text,standardized_text,no_sw_text,lemmatized_sw_text,lemmatized_no_sw_text
project_code,,,,,,,,,
2018.1.01205.L,Fifty AU STudy of the chemistry in the disk/en...,The huge variety of planetary systems discover...,line,1,Fifty AU STudy of the chemistry in the disk/en...,fifty au study of the chemistry in the disk en...,fifty au study chemistry disk envelope system ...,fifty au study of the chemistry in the disk en...,fifty au study chemistry disk envelope system ...
2022.1.00316.L,COMPASS: Complex Organic Molecules in Protosta...,The emergence of complex organic molecules in ...,line,1,COMPASS: Complex Organic Molecules in Protosta...,compass complex organic molecules in protostar...,compass complex organic molecules protostars s...,compass complex organic molecule in protostars...,compass complex organic molecule protostars sp...
2017.1.00161.L,ALCHEMI: the ALMA Comprehensive High-resolutio...,A great variety in gas composition is observed...,line,1,ALCHEMI: the ALMA Comprehensive High-resolutio...,alchemi the comprehensive high resolution extr...,alchemi comprehensive high resolution extragal...,alchemi the comprehensive high resolution extr...,alchemi comprehensive high resolution extragal...
2021.1.01616.L,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,We propose the first ever statistical survey o...,line,1,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,jelly survey of nearby jellyfish and ram press...,jelly survey nearby jellyfish ram pressure str...,jelly survey of nearby jellyfish and ram press...,jelly survey nearby jellyfish ram pressure str...
2021.1.00869.L,Bulge symmetry or not? The hidden dynamics of ...,A radio survey of red giant SiO sources in the...,line,1,Bulge symmetry or not? The hidden dynamics of ...,bulge symmetry or not the hidden dynamics of t...,bulge symmetry hidden dynamics far side radio ...,bulge symmetry or not the hidden dynamic of th...,bulge symmetry hidden dynamic far side radio s...


In [4]:
line_projects = projects.query('fs_type == "line"')
line_projects.shape

(3628, 9)

In [5]:
measurements = pd.read_csv('../../nrao_measurements.csv')
measurements = measurements.set_index('project_code')

In [6]:
measurements.head()

,project_title,project_abstract,fs_type,low_freq,high_freq,target,diff_freq,med_freq,raw_text,standardized_text,no_sw_text,lemmatized_sw_text,lemmatized_no_sw_text
project_code,,,,,,,,,,,,,
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.38,90.62,1,0.24,90.500,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.70,90.93,1,0.23,90.815,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,91.69,91.92,1,0.23,91.805,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,92.89,93.12,1,0.23,93.005,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,217.59,218.53,1,0.94,218.060,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...


## Train-test split

In [7]:
train_texts, test_texts = train_test_split(line_projects.lemmatized_no_sw_text, random_state=SEED)

In [8]:
print(f'Number of train texts:{len(list(train_texts))}')
print(f'Number of test texts:{len(list(test_texts))}')

Number of train texts:2721
Number of test texts:907


In [9]:
train_texts

project_code
2017.1.00065.S    co dark molecular gas extend ultraviolet disk ...
2013.1.00530.S    detailed view star form gas dusty herschel atl...
2016.1.00243.S    confirm deeply embed protostellar population c...
2021.2.00056.S    panta rei mass energy flow parsec sub parsec s...
2021.1.00347.S    trigger supression star formation recent gas r...
                                        ...                        
2015.1.00315.S    secret live badgrs discover nearby population ...
2016.1.00949.S    uncover early formation extremely massive star...
2013.1.00448.S    origin warm water emission low mass protostars...
2023.1.00750.S    hii gas cosmic noon aca survey oiii z aim surv...
2021.2.00140.S    origin rotational motion dense core rotation s...
Name: lemmatized_no_sw_text, Length: 2721, dtype: object

### LDA class

In [10]:
class LDA_Model:
    def __init__(self, N_topics=3):
        self.N_topics = N_topics
        self.countVectorizer = CountVectorizer(stop_words='english')
        self.lda = LatentDirichletAllocation(n_components=self.N_topics, random_state=SEED)
    
    def fit(self, corpus):
        termFrequency = self.countVectorizer.fit_transform(corpus)
        self.lda.fit(termFrequency)
        return self.lda.transform(termFrequency)

    # Additional method to transform new data
    def transform(self, corpus):
        termFrequency = self.countVectorizer.transform(corpus)
        return self.lda.transform(termFrequency)

#### Initialize Model

In [11]:
#lda_model = LDA_Model(N_topics=50)
lda_model = LDA_Model(N_topics=200)

#### Fit model on training set

In [12]:
train_topics = lda_model.fit(train_texts)

In [13]:
words = lda_model.countVectorizer.get_feature_names_out()

In [14]:
N = 10 #number of top words to show
topic_components = lda_model.lda.components_

for topic_idx, topic in enumerate(topic_components):
    print(f"Topic {topic_idx}:")
    # Get the indices of the top N words for this topic
    top_word_indices = topic.argsort()[-N:][::-1]
    # Print these words with their weights
    for word_idx in top_word_indices:
        print(f"{words[word_idx]} (weight: {topic[word_idx]:.2f})")
    print("\n")

Topic 0:
star (weight: 53.15)
mass (weight: 37.58)
survey (weight: 34.14)
disk (weight: 31.10)
high (weight: 27.45)
au (weight: 26.75)
protostars (weight: 25.37)
outflow (weight: 22.05)
resolution (weight: 21.55)
region (weight: 20.87)


Topic 1:
big (weight: 17.28)
quasar (weight: 17.26)
growth (weight: 13.17)
bang (weight: 11.80)
example (weight: 9.65)
gyr (weight: 7.65)
regime (weight: 6.88)
arcsec (weight: 5.59)
myr (weight: 5.16)
new (weight: 4.40)


Topic 2:
shock (weight: 22.23)
observation (weight: 17.22)
propose (weight: 16.54)
emission (weight: 16.42)
sio (weight: 14.74)
band (weight: 12.73)
region (weight: 11.88)
molecular (weight: 11.33)
disk (weight: 9.21)
data (weight: 8.54)


Topic 3:
gas (weight: 48.78)
disk (weight: 47.98)
sulphur (weight: 14.00)
observation (weight: 13.51)
neutral (weight: 13.40)
protoplanetary (weight: 10.77)
propose (weight: 10.25)
suggest (weight: 9.47)
large (weight: 9.08)
ion (weight: 9.00)


Topic 4:
disk (weight: 115.62)
star (weight: 55.07)
ro

In [15]:
train_doc_topic = pd.DataFrame(train_topics)
train_doc_topic = train_doc_topic.set_index(train_texts.index.values)
train_doc_topic.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
2017.1.00065.S,0.000045,0.000045,0.000045,0.000045,0.000045,0.000045,0.000045,0.000045,0.000045,0.029878,...,0.000045,0.000045,0.000045,0.000045,0.000045,0.000045,0.000045,0.000045,0.000045,0.197127
2013.1.00530.S,0.000122,0.000122,0.000122,0.000122,0.000122,0.000122,0.000122,0.000122,0.000122,0.000122,...,0.000122,0.000122,0.000122,0.000122,0.000122,0.000122,0.000122,0.000122,0.000122,0.000122
2016.1.00243.S,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,...,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042,0.000042
2021.2.00056.S,0.000049,0.000049,0.000049,0.000049,0.000049,0.000049,0.000049,0.000049,0.000049,0.000049,...,0.000049,0.000049,0.000049,0.000049,0.000049,0.000049,0.000049,0.000049,0.000049,0.000049
2021.1.00347.S,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,...,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041,0.000041


In [16]:
train_texts = pd.DataFrame(train_texts)

### Match test data into topics

In [17]:
test_topics = lda_model.transform(test_texts)

In [18]:
test_doc_topic= pd.DataFrame(test_topics.tolist())
test_doc_topic= test_doc_topic.set_index(test_texts.index.values)
test_doc_topic.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
2015.1.01599.S,0.000057,0.000057,0.000057,0.000057,0.000057,0.000057,0.000057,0.000057,0.000057,0.000057,...,0.000057,0.000057,0.000057,0.000057,0.000057,0.000057,0.000057,0.000057,0.000057,0.000057
2017.1.01119.S,0.000054,0.000054,0.000054,0.000054,0.000054,0.000054,0.000054,0.000054,0.000054,0.000054,...,0.024059,0.000054,0.000054,0.000054,0.000054,0.000054,0.000054,0.000054,0.000054,0.000054
2018.1.01172.S,0.000069,0.000069,0.000069,0.000069,0.000069,0.000069,0.000069,0.000069,0.000069,0.425351,...,0.000069,0.000069,0.000069,0.000069,0.000069,0.000069,0.000069,0.000069,0.000069,0.000069
2022.1.00181.S,0.015818,0.000046,0.000046,0.000046,0.043780,0.000046,0.000046,0.000046,0.000046,0.000046,...,0.000046,0.000046,0.000046,0.000046,0.000046,0.000046,0.072050,0.000046,0.046621,0.000046
2019.2.00190.S,0.000043,0.000043,0.000043,0.000043,0.000043,0.370751,0.000043,0.000043,0.000043,0.000043,...,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043,0.000043


In [19]:
test_texts = pd.DataFrame(test_texts)

## Try comparing two projects based on topic vector similarity using dot product

In [20]:
comp = np.dot(train_doc_topic, test_doc_topic.iloc[1].T)

In [21]:
train_doc_topic.iloc[comp.argmax()].name

'2011.0.00656.S'

In [22]:
comp_frame = pd.DataFrame(comp)\
    .set_index(train_texts.index.values)
comp_frame.columns = ['dot_product_similarity']
comp_frame = comp_frame.sort_values('dot_product_similarity', ascending=False)

In [23]:
comp_frame.shape

(2721, 1)

In [24]:
comp_frame.describe()

,dot_product_similarity
count,2721.000000
mean,0.009378
std,0.019886
min,0.000071
25%,0.000100
50%,0.000111
75%,0.008654
max,0.159787


In [25]:
comp_frame.dot_product_similarity.quantile(q=0.75)

0.008654267382427947

In [26]:
comp_frame.head()

,dot_product_similarity
2011.0.00656.S,0.159787
2017.1.01214.S,0.153761
2012.1.00719.S,0.152414
2019.2.00053.S,0.148272
2021.2.00064.S,0.147066


In [27]:
q4_comp = comp_frame[comp_frame.dot_product_similarity > comp_frame.dot_product_similarity.quantile(q=0.999)]
q4_comp

,dot_product_similarity
2011.0.00656.S,0.159787
2017.1.01214.S,0.153761
2012.1.00719.S,0.152414


In [28]:
px.histogram(q4_comp.dot_product_similarity)

In [29]:
train_texts.iloc[1896]

lemmatized_no_sw_text    smc b new perseus relationship av co low metal...
Name: 2015.1.01125.S, dtype: object

In [30]:
test_texts.iloc[0]

lemmatized_no_sw_text    remote detection organic enceladus plume encel...
Name: 2015.1.01599.S, dtype: object

In [31]:
fig = px.scatter(measurements.loc[list(q4_comp.index.values)].med_freq)
#fig.add_trace(px.scatter(measurements.loc['2017.1.01247.S'].med_freq).data[0])
fig

## Loop over test points and find differences between frequency setup for test point and top (ties not resolved...maybe use most recent?) dot product similarity project 

In [32]:
proj_measurements = measurements[['low_freq', 'high_freq', 'diff_freq', 'med_freq']]
proj_measurements = proj_measurements.groupby(proj_measurements.index).agg({
    'low_freq':lambda x: round(x, 4).tolist(),
    'high_freq':lambda x: round(x, 4).tolist(),
    'diff_freq':lambda x: round(x, 4).tolist(),
    'med_freq':lambda x: round(x, 4).tolist()
})
proj_measurements

,low_freq,high_freq,diff_freq,med_freq
project_code,,,,
2011.0.00010.S,"[90.38, 90.7, 91.69, 92.89, 217.59, 218.67, 21...","[90.62, 90.93, 91.92, 93.12, 218.53, 219.6, 21...","[0.24, 0.23, 0.23, 0.23, 0.94, 0.93, 0.94, 0.9...","[90.5, 90.815, 91.805, 93.005, 218.06, 219.135..."
2011.0.00017.S,"[87.72, 89.54, 99.72, 101.54, 91.37, 93.19, 10...","[89.6, 91.42, 101.59, 103.42, 93.24, 95.07, 10...","[1.88, 1.88, 1.87, 1.88, 1.87, 1.88, 1.88, 1.8...","[88.66, 90.48, 100.655, 102.48, 92.305, 94.13,..."
2011.0.00020.S,"[334.61, 346.72, 328.56, 330.44, 340.63, 342.3...","[336.5, 348.61, 330.44, 332.32, 342.51, 344.27...","[1.89, 1.89, 1.88, 1.88, 1.88, 1.88, 1.92, 1.9...","[335.555, 347.665, 329.5, 331.38, 341.57, 343...."
2011.0.00028.S,"[342.36, 344.24, 354.28, 355.79, 342.36]","[344.23, 346.11, 356.16, 357.66, 344.24]","[1.87, 1.87, 1.88, 1.87, 1.88]","[343.295, 345.175, 355.22, 356.725, 343.3]"
2011.0.00034.S,"[84.13, 85.83, 96.18, 97.69]","[86.11, 87.81, 98.17, 99.67]","[1.98, 1.98, 1.99, 1.98]","[85.12, 86.82, 97.175, 98.68]"
...,...,...,...,...
2023.1.01719.S,"[89.53, 91.49, 101.53, 103.49]","[91.51, 93.47, 103.51, 105.47]","[1.98, 1.98, 1.98, 1.98]","[90.52, 92.48, 102.52, 104.48]"
2023.1.01720.S,"[216.99, 217.88, 220.28, 220.88, 231.56, 234.0...","[217.22, 218.12, 220.52, 221.12, 233.44, 235.9...","[0.23, 0.24, 0.24, 0.24, 1.88, 1.88, 1.98, 1.9...","[217.105, 218.0, 220.4, 221.0, 232.5, 235.0, 9..."
2023.1.01721.S,"[340.64, 342.52, 352.66, 354.53]","[342.52, 344.39, 354.54, 356.41]","[1.88, 1.87, 1.88, 1.88]","[341.58, 343.455, 353.6, 355.47]"


In [33]:
proj_measurements.low_freq = proj_measurements.low_freq.apply(lambda x: sorted(x))
proj_measurements.high_freq = proj_measurements.high_freq.apply(lambda x: sorted(x))

In [34]:
proj_measurements.head()

,low_freq,high_freq,diff_freq,med_freq
project_code,,,,
2011.0.00010.S,"[85.96, 87.05, 87.47, 87.65, 88.36, 88.91, 90....","[86.19, 87.29, 87.7, 87.88, 88.59, 89.15, 90.6...","[0.24, 0.23, 0.23, 0.23, 0.94, 0.93, 0.94, 0.9...","[90.5, 90.815, 91.805, 93.005, 218.06, 219.135..."
2011.0.00017.S,"[84.09, 85.9, 87.72, 89.54, 91.37, 93.19, 95.0...","[85.97, 87.78, 89.6, 91.42, 93.24, 95.07, 96.9...","[1.88, 1.88, 1.87, 1.88, 1.87, 1.88, 1.88, 1.8...","[88.66, 90.48, 100.655, 102.48, 92.305, 94.13,..."
2011.0.00020.S,"[251.53, 253.41, 257.24, 266.22, 267.87, 272.3...","[253.42, 255.29, 259.13, 268.11, 269.75, 274.2...","[1.89, 1.89, 1.88, 1.88, 1.88, 1.88, 1.92, 1.9...","[335.555, 347.665, 329.5, 331.38, 341.57, 343...."
2011.0.00028.S,"[342.36, 342.36, 344.24, 354.28, 355.79]","[344.23, 344.24, 346.11, 356.16, 357.66]","[1.87, 1.87, 1.88, 1.87, 1.88]","[343.295, 345.175, 355.22, 356.725, 343.3]"
2011.0.00034.S,"[84.13, 85.83, 96.18, 97.69]","[86.11, 87.81, 98.17, 99.67]","[1.98, 1.98, 1.99, 1.98]","[85.12, 86.82, 97.175, 98.68]"


In [35]:
best_matches = {}
for proj in test_doc_topic.index.values:
    dot_product = np.dot(train_doc_topic, test_doc_topic.loc[proj].T)
    best_matches[proj] = train_doc_topic.iloc[dot_product.argmax()].name

In [36]:
best_matches_frame = pd.DataFrame.from_dict(best_matches, orient='index')
best_matches_frame.columns = ['best_match']
best_matches_frame = best_matches_frame.reset_index()
best_matches_frame = best_matches_frame.set_index('index')
best_matches_frame.index = best_matches_frame.index.rename('test_project')
best_matches_frame.head()

,best_match
test_project,
2015.1.01599.S,2017.1.01311.S
2017.1.01119.S,2011.0.00656.S
2018.1.01172.S,2019.1.00500.S
2022.1.00181.S,2022.1.00810.S
2019.2.00190.S,2017.1.00303.S


In [37]:
best_matches_frame.describe()

,best_match
count,907
unique,177
top,2013.1.00530.S
freq,85


In [38]:
best_matches_frame.value_counts().describe()

count    177.000000
mean       5.124294
std        9.591446
min        1.000000
25%        1.000000
50%        2.000000
75%        5.000000
max       85.000000
Name: count, dtype: float64

In [39]:
train_doc_topic.loc['2017.1.00815.S']

0      0.000048
1      0.000048
2      0.000048
3      0.000048
4      0.000048
         ...   
195    0.000048
196    0.000048
197    0.000048
198    0.000048
199    0.076474
Name: 2017.1.00815.S, Length: 200, dtype: float64

In [40]:
best_matches_frame[best_matches_frame.best_match == '2017.1.00815.S']

,best_match
test_project,


## Compare best match frequency setups

In [41]:
def find_common_elements(list1, list2) -> list:
    set1 = set(list1)
    set2 = set(list2)
    
    return list(set1.intersection(set2))

In [42]:
def compare_matches(matches_frame:pd.DataFrame, comp_column:str, round:int) -> list:
    percent_test_match_list = []
    for test_proj in matches_frame.index:
        match_proj = matches_frame.loc[test_proj].best_match
        test_measurements = np.round(proj_measurements.loc[test_proj][comp_column], round)
        match_measurements = np.round(proj_measurements.loc[match_proj][comp_column], round)
        common_elements = find_common_elements(test_measurements, match_measurements)
        percent_test_match = len(common_elements)/len(test_measurements)
        percent_test_match_list.append(percent_test_match)
        print(f'Test project {test_proj} compared to best match {match_proj}')
        print(f'len(matched measurements)/len(test measurements) = {percent_test_match}')
    return percent_test_match_list

In [43]:
np.mean(compare_matches(best_matches_frame, 'med_freq', 0))

Test project 2015.1.01599.S compared to best match 2017.1.01311.S
len(matched measurements)/len(test measurements) = 0.0
Test project 2017.1.01119.S compared to best match 2011.0.00656.S
len(matched measurements)/len(test measurements) = 0.0
Test project 2018.1.01172.S compared to best match 2019.1.00500.S
len(matched measurements)/len(test measurements) = 0.0
Test project 2022.1.00181.S compared to best match 2022.1.00810.S
len(matched measurements)/len(test measurements) = 0.0
Test project 2019.2.00190.S compared to best match 2017.1.00303.S
len(matched measurements)/len(test measurements) = 0.0
Test project 2019.A.00019.S compared to best match 2017.1.01214.S
len(matched measurements)/len(test measurements) = 0.0
Test project 2015.1.01199.S compared to best match 2021.1.00914.S
len(matched measurements)/len(test measurements) = 0.3333333333333333
Test project 2018.1.01506.S compared to best match 2017.1.00893.S
len(matched measurements)/len(test measurements) = 0.4666666666666667
Te

0.09868324840911075

In [44]:
for test_proj in best_matches_frame.index:
    print(best_matches_frame.loc[test_proj].best_match)

2017.1.01311.S
2011.0.00656.S
2019.1.00500.S
2022.1.00810.S
2017.1.00303.S
2017.1.01214.S
2021.1.00914.S
2017.1.00893.S
2017.1.00727.S
2022.1.00860.S
2015.1.00389.S
2013.1.00166.S
2017.1.00303.S
2017.1.01019.S
2015.1.01486.S
2015.1.01093.S
2016.1.00223.S
2017.1.01214.S
2017.1.01109.S
2012.1.01069.S
2021.1.01342.S
2016.2.00006.S
2017.1.01019.S
2015.1.01093.S
2019.1.00743.S
2018.A.00023.S
2017.1.01214.S
2017.1.00727.S
2015.1.01093.S
2013.1.00530.S
2021.2.00123.S
2019.1.01317.S
2013.1.00530.S
2019.1.00566.S
2019.1.00566.S
2012.1.00060.S
2019.1.00949.S
2013.1.01365.S
2019.1.00949.S
2017.1.01214.S
2022.1.01439.S
2023.1.01346.S
2016.1.00051.S
2017.1.00727.S
2016.1.01154.V
2011.0.00525.S
2017.1.01214.S
2019.1.00500.S
2017.1.01424.S
2015.1.00656.S
2021.1.01342.S
2017.1.00727.S
2013.1.00530.S
2013.1.00166.S
2013.1.00379.S
2022.1.01439.S
2015.1.00449.S
2013.1.00166.S
2017.1.00727.S
2017.1.01404.S
2017.1.00914.S
2017.1.01214.S
2017.1.00727.S
2018.1.00397.S
2019.1.01317.S
2021.1.00162.S
2018.1.016